In [2]:
import torch
import torch.onnx
import onnx
import onnxruntime as rt
import numpy as np
import pandas as pd
# import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pathlib import Path

In [2]:
DATASET_DIR = '/home/italolanza/workspace/TG/dataset/'
DATASET_NAME = 'dataset_completo.csv' 
DATASET_PATH = DATASET_DIR+DATASET_NAME
MODEL_DIR = '/home/italolanza/workspace/TG/graduation-project/models/'

In [13]:
collum_name = ['Tacom_1f0', 'Tacom_2f0', 'Tacom_3f0', 'Tacom_kurtosis', 'Tacom_entropy', 'Aceler_Underhang_X_1f0', 'Aceler_Underhang_X_2f0', 'Aceler_Underhang_X_3f0', 'Aceler_Underhang_X_kurtosis', 'Aceler_Underhang_X_entropy', 'Aceler_Underhang_Y_1f0', 'Aceler_Underhang_Y_2f0', 'Aceler_Underhang_Y_3f0', 'Aceler_Underhang_Y_kurtosis', 'Aceler_Underhang_Y_entropy', 'Aceler_Underhang_Z_1f0', 'Aceler_Underhang_Z_2f0', 'Aceler_Underhang_Z_3f0', 'Aceler_Underhang_Z_kurtosis', 'Aceler_Underhang_Z_entropy', 'Aceler_Overhang_X_1f0', 'Aceler_Overhang_X_2f0', 'Aceler_Overhang_X_3f0', 'Aceler_Overhang_X_kurtosis', 'Aceler_Overhang_X_entropy', 'Aceler_Overhang_Y_1f0', 'Aceler_Overhang_Y_2f0', 'Aceler_Overhang_Y_3f0', 'Aceler_Overhang_Y_kurtosis', 'Aceler_Overhang_Y_entropy', 'Aceler_Overhang_Z_1f0', 'Aceler_Overhang_Z_2f0', 'Aceler_Overhang_Z_3f0', 'Aceler_Overhang_Z_kurtosis', 'Aceler_Overhang_Z_entropy', 'Audio_1f0', 'Audio_2f0', 'Audio_3f0', 'Audio_kurtosis', 'Audio_entropy', 'Class']
all_features = ['Tacom_1f0', 'Tacom_2f0', 'Tacom_3f0', 'Tacom_kurtosis', 'Tacom_entropy', 'Aceler_Underhang_X_1f0', 'Aceler_Underhang_X_2f0', 'Aceler_Underhang_X_3f0', 'Aceler_Underhang_X_kurtosis', 'Aceler_Underhang_X_entropy', 'Aceler_Underhang_Y_1f0', 'Aceler_Underhang_Y_2f0', 'Aceler_Underhang_Y_3f0', 'Aceler_Underhang_Y_kurtosis', 'Aceler_Underhang_Y_entropy', 'Aceler_Underhang_Z_1f0', 'Aceler_Underhang_Z_2f0', 'Aceler_Underhang_Z_3f0', 'Aceler_Underhang_Z_kurtosis', 'Aceler_Underhang_Z_entropy', 'Aceler_Overhang_X_1f0', 'Aceler_Overhang_X_2f0', 'Aceler_Overhang_X_3f0', 'Aceler_Overhang_X_kurtosis', 'Aceler_Overhang_X_entropy', 'Aceler_Overhang_Y_1f0', 'Aceler_Overhang_Y_2f0', 'Aceler_Overhang_Y_3f0', 'Aceler_Overhang_Y_kurtosis', 'Aceler_Overhang_Y_entropy', 'Aceler_Overhang_Z_1f0', 'Aceler_Overhang_Z_2f0', 'Aceler_Overhang_Z_3f0', 'Aceler_Overhang_Z_kurtosis', 'Aceler_Overhang_Z_entropy', 'Audio_1f0', 'Audio_2f0', 'Audio_3f0', 'Audio_kurtosis', 'Audio_entropy']

In [8]:
DATASET_BASE_DIR = Path("/home/italolanza/workspace/TG/dataset/")
VALIDATION_INPUT_DATA = DATASET_BASE_DIR.joinpath('x_test.csv')
VALIDATION_OUTPUT_DATA = DATASET_BASE_DIR.joinpath('y_test.csv')

print(VALIDATION_INPUT_DATA)
print(VALIDATION_OUTPUT_DATA)


x_data = np.loadtxt(VALIDATION_INPUT_DATA,delimiter=',')
y_data = np.loadtxt(VALIDATION_OUTPUT_DATA,delimiter=',')

print(x_data[0:2])
print(y_data)

/home/italolanza/workspace/TG/dataset/x_test.csv
/home/italolanza/workspace/TG/dataset/y_test.csv
[[ 9.88460975e+04  1.43144546e+05  8.02879299e+04  3.10412119e+00
  -1.36940581e+01  2.29628759e+04  7.05134046e+03  1.75745122e+04
  -6.67190080e-01 -5.40683139e+00  1.00083939e+05  1.06844394e+04
   4.38800077e+03 -1.17755927e+00 -5.36533633e+00  1.84558197e+04
   1.51452283e+03  2.13790247e+03 -5.62914583e-01 -8.87194006e+00
   1.33628975e+04  6.12351348e+03  1.66809585e+03 -1.49322344e-01
  -7.20594230e+00  8.94790661e+04  7.58171310e+04  1.07648973e+04
  -7.09201014e-01 -4.64113509e+00  3.02847543e+04  1.62509932e+04
   3.19085611e+03 -3.86140248e-01 -5.14881998e+00  1.67967722e+03
   4.14992433e+02  1.70166689e+03 -1.99701358e-01 -1.80275871e+01]
 [ 1.37014348e+05  1.23941583e+05  1.04247215e+05  4.05778204e+00
  -1.37617420e+01  2.65863746e+04  2.07713904e+03  2.22841045e+04
  -9.22568337e-01 -7.00796080e+00  1.64708297e+03  4.68948168e+03
   1.60697666e+03 -8.23566497e-01 -6.130606